In [76]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import math
import copy

# Multi-Head Attention

In [77]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()

        # number of heads should be a multiple of dimension of d_model(input, hidden layers and output)
        assert d_model%num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model # dimension of hidden layers, input and output
        self.num_heads = num_heads # number of heads in multiheaded attention
        self.d_k = d_model//num_heads # dimention of Querues and Keys after linear projection into num_heads

        self.W_q = nn.Linear(d_model, d_model) # weight matrix of Queries
        self.W_k = nn.Linear(d_model, d_model) # weight matrix of Keys
        self.W_v = nn.Linear(d_model, d_model) # weight matrix of Values
        self.W_o = nn.Linear(d_model, d_model) # weight matrix of Output

    # calculate scaled dot production attention score
    def scaled_dot_product_attention(self, Q, K, V, mask = None):

        # scaling dot product of Q and K by sqrt(d_k)
        attn_scores = torch.matmul(Q, K.transpose(-2, -1))/math.sqrt(self.d_k)

        # mask illegal tokens
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask==0, -1e9)

        # transform attention score to probablities using softmax function
        attn_probs = torch.softmax(attn_scores, dim=-1)

        # weighted sum of values
        output = torch.matmul(attn_probs, V)

        return output
    
    # split attention into multiple heads
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1,2)
    
    # concatenated multiple heads into one
    def combine_heads(self, x):
        batch_size, _, seq_lenght, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_lenght, self.d_model)
    
    def forward(self, Q, K, V, mask=None):

        Q = self.split_heads(self.W_q(Q)) # calculate query
        K = self.split_heads(self.W_k(K)) # calculate keys
        V = self.split_heads(self.W_v(V)) # calculate values

        # calculate attention values for multiple heads
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)

        # combine attention values from multiple head and final attention output
        output = self.W_o(self.combine_heads(attn_output))
        return output



# Position Wise Feed Foreward Neural Network

In [78]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        output = self.fc1(x)
        output = nn.ReLU()(output)
        output = self.fc2(output)
        return output

# Encoder and Decoder

In [79]:
class Encoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        # attention layer
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        # feed forward neural network
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        # layer norm after attendtion
        self.norm1 = nn.LayerNorm(d_model)
        # layer norm after feed forward
        self.norm2 = nn.LayerNorm(d_model)
        # dropout layer after output of each sublayer layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask) # residual connection
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))# residual connection
        return x
    
class Decoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(Decoder, self).__init__()
        # self attention layer
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        # encoder decoder attention layer
        self.encoder_decoder_attn = MultiHeadAttention(d_model, num_heads)
        # Feed forwards layer
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        # layer norm after self attn
        self.norm1 = nn.LayerNorm(d_model)
        # layer norm after encoder decoder attn
        self.norm2 = nn.LayerNorm(d_model)
        # layer norm after feed forward neural network
        self.norm3 = nn.LayerNorm(d_model)
        # drpout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        self_attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x+self.dropout(self_attn_output)) # residual connection 
        enc_dec_attn_output = self.encoder_decoder_attn(x, encoder_output, encoder_output, src_mask)
        x = self.norm2(x+self.dropout(enc_dec_attn_output)) # residual connection
        ff_output = self.feed_forward(x)
        x = self.norm3(x+self.dropout(ff_output)) # residula connection
        return x


# Positional Encoding


In [80]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000)))

        pe[:, 0::2] = torch.sin(position*div_term)
        pe[:, 1::2] = torch.cos(position*div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x+self.pe[:, :x.size(1)]

# Transformer Architecture

In [81]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()

        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_stack = nn.ModuleList([Encoder(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_stack = nn.ModuleList([Decoder(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(2)
        seq_length = tgt.size(1)
        nopeak_mask = (1-torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask
    
    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embeddings = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_emneddings = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embeddings
        for enc_layer in self.encoder_stack:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_emneddings
        for dec_layer in self.decoder_stack:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)

        return output



# Prepare Sample Data

In [82]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length)) 

# Training the model

In [83]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.68905258178711
Epoch: 2, Loss: 8.563027381896973
Epoch: 3, Loss: 8.495355606079102
Epoch: 4, Loss: 8.447508811950684
Epoch: 5, Loss: 8.403788566589355
Epoch: 6, Loss: 8.347920417785645
Epoch: 7, Loss: 8.283411979675293
Epoch: 8, Loss: 8.198308944702148
Epoch: 9, Loss: 8.12839126586914
